## Properties / Metadata
The document properties represent the fields that define the data elements you want to retrieve as well as the elements you choose to filter your results.  In addition, properties are at the heart of other Search parameters such as Navigation, Grouping, and Boosting.  There are hundreds of properties available for each View providing critical details needed to fulfill your searching requirements.

In order for you to be successful at search, you will need to understand how to access the available properties, how to determine what values they contain and what they mean.

**Note**: The concepts outlined within this notebook provide the mechanism to manually extract properties and values.  While the techniques outlined are a great tool in understanding
      how to better understand these attributes, the tasks to extract these details can be cumbersome.  As a way to significantly reduce this effort, I've
      created a [convenient package](https://github.com/Refinitiv-API-Samples/Article.RDPLibrary.Python.SearchBrowser) that will allow users to discover properties and values much easier when searching.  This utility piggybacks off of the techniques outlined here.

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd

# Default session - desktop
rd.open_session(app_key='Your API Key here')

<refinitiv.data.session.Definition object at 0x1d212b81220 {name='default'}>

In [2]:
pd.set_option('display.max_colwidth', 140)
rd.__version__

'1.0.0b9'

In [3]:
# Optionally override maximum dataframe rows
#pd.set_option('display.max_rows', 10000)

In [4]:
help(search.SearchViews)

Help on class SearchViews in module refinitiv.data._data.content.search.search_view_type:

class SearchViews(enum.Enum)
 |  SearchViews(value, names=None, *, module=None, qualname=None, type=None, start=1)
 |  
 |  Possible views values to request data from 'search' endpoint
 |  
 |  Method resolution order:
 |      SearchViews
 |      enum.Enum
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  BOND_FUT_OPT_QUOTES = <SearchViews.BOND_FUT_OPT_QUOTES: 'BondFutOptQuo...
 |  
 |  CDS_INSTRUMENTS = <SearchViews.CDS_INSTRUMENTS: 'CdsInstruments'>
 |  
 |  CDS_QUOTES = <SearchViews.CDS_QUOTES: 'CdsQuotes'>
 |  
 |  CMO_INSTRUMENTS = <SearchViews.CMO_INSTRUMENTS: 'CmoInstruments'>
 |  
 |  CMO_QUOTES = <SearchViews.CMO_QUOTES: 'CmoQuotes'>
 |  
 |  COMMODITY_QUOTES = <SearchViews.COMMODITY_QUOTES: 'CommodityQuotes'>
 |  
 |  DEALS_MERGERS_AND_ACQUISITIONS = <SearchViews.DEALS_MERGERS_AND_ACQUIS...
 |  
 |  DERIVATIVE_INSTRUMENTS = <SearchViews.DERIVATIVE_INSTRUM

#### Metadata
MetaData is a convenient service that allows you to list all available properties within a View.  To properly build a metadata list for inspection, you can issue the following:

In [5]:
# Specify the view
response = search.metadata.Definition(
    view = search.SearchViews.COMMODITY_QUOTES
).get_data()
response.data.df

,Type,Searchable,Sortable,Navigable,Groupable,Exact,Symbol
AAACurrencyBondBenchmarkChain,String,False,False,False,False,False,False
AACurrencyBondBenchmarkChain,String,False,False,False,False,False,False
ABSMBSBondsRIC,String,False,False,False,False,False,False
ActivityDate,Date,True,True,True,False,False,False
ACurrencyBondBenchmarkChain,String,False,False,False,False,False,False
...,...,...,...,...,...,...,...
WarrantsCount,Double,True,True,True,False,False,False
WarrantType,String,True,False,False,False,False,False
WarrantTypeName,String,True,False,False,False,False,False
Wert,String,True,False,False,True,False,True


In [7]:
# Export metadata to a spreadsheet for easy viewing
response.data.df.to_excel("CommodityQuotes.xlsx")

#### Debugging (*_debugall*)
While MetaData will help list all properties available within a View, all properties will not be available for use.  That is, depending on the type of search you perform, will determine which properties will be populated - not all will be.  This is where users can use a technique to list all available properties that are populated, for the specific search requested.

In [6]:
# Query for all documents with the company IBM defined
response=search.Definition(
    query = "IBM",
    top = 1,    # Note: It is critical you select a top=1 because the amount of data returned is very large.
    select = "_debugall"
).get_data()
response.data.raw

{'Total': 45372,
 'Hits': [{'meta': {'_id': '37036',
    '_index': 'eikonsearch_organisation_1.139.0',
    '_score': 2838801.0,
    '_timestamp': 1648202085085,
    '_timestamp_human': '2022-03-25T09:54:45.085Z',
    '_version': 1648201134000},
   'raw_source': {'RCSOrganisationTypeGenealogy': 'OT:1',
    'SPsRatingOutlookX1XSPsRatingOutlookRank': 'SPI/StableX1X0|SPI/StableX1X0',
    'RCSTRBCGenealogy': 'B:161\\B:171\\B:172\\B:173',
    'IssuerID': '0x000019000050e3aa',
    'SicSchemeCode': '8742',
    'OfficersExist': True,
    'IssuerBondsExist': True,
    'CertificatesOfDepositCount': 0,
    'MktCapCompanyUsd': 115381471203,
    'TotalAssetsUsd': 132001000000,
    'GEMPermID': '4295904307',
    'SubjectBoost': 'firm',
    'EEDBID': '14129',
    'SigDevExist': True,
    'SDCID': '3104001',
    'TotalAssets': 132001000000,
    'EstimatesExist': True,
    'Ticker': 'IBM',
    'CdsUltimateParentCount': 0,
    'FilingsExist': True,
    'CompanyOfficers': 'Senior Vice President:Erich Clem

#### Understanding values using Navigators
Using the output from a debugging session, there are a number of common properties that are very useful when filtering your search.  For example, you may want to filter your results based on a specific category of data.  Using navigators will help us understand these categories.

Looking at our debug output above, one useful property: **SearchAllCategoryv3** will helps us understand all available categories of documents the query term 'IBM' was found.  To list these categories, use the following:

In [7]:
response=search.Definition(
    query = "IBM",
    top = 0,
    navigators = "SearchAllCategoryv3"
).get_data()
response.data.raw["Navigators"]["SearchAllCategoryv3"]["Buckets"]

[{'Label': 'Other Quote', 'Count': 26225},
 {'Label': 'Warrants', 'Count': 8633},
 {'Label': 'Bond Pricing', 'Count': 2169},
 {'Label': 'Bonds', 'Count': 1947},
 {'Label': 'Options', 'Count': 1372},
 {'Label': 'Commercial Papers', 'Count': 1310},
 {'Label': 'Futures', 'Count': 769},
 {'Label': 'Companies/Issuers', 'Count': 747},
 {'Label': 'Mergers & Acquisitions', 'Count': 732},
 {'Label': 'Equities', 'Count': 546},
 {'Label': 'Credit Default Swaps', 'Count': 281},
 {'Label': 'Analyst', 'Count': 241},
 {'Label': 'Officers', 'Count': 202},
 {'Label': 'FX & Money', 'Count': 65},
 {'Label': 'Fixed Income Curves', 'Count': 48},
 {'Label': 'Other Instrument', 'Count': 35},
 {'Label': 'Mortgage Backed Securities', 'Count': 12},
 {'Label': 'Funds', 'Count': 10},
 {'Label': 'People', 'Count': 7},
 {'Label': 'CMO & ABS', 'Count': 6},
 {'Label': 'Buy Side Contact', 'Count': 4},
 {'Label': 'Fund Manager', 'Count': 4},
 {'Label': 'Loans', 'Count': 4},
 {'Label': 'Commodities Physical Assets', 'Co

Using the above listing, we can narrow our result set based on these categories using a Filter.

#### Special Properties
The Search service includes a number of special properties that enhance the ability to find meaningful results. In addition to the special property *_debugall* used within the **select** parameter as outlined above, the following properties are available: 

In [8]:
# The 'code' specification will search across all codes, such as RICs, tickers, SEDOLs etc.  
# Primarily used within a filter but can be used as a selectable property.
response=search.Definition(
    filter = "code eq 'IBM'"
).get_data()
response.data.df

,BusinessEntity,DocumentTitle,PermID,PI,RIC
0,ORGANISATION,"International Business Machines Corp, Public Company",<NA>,37036,<NA>
1,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",55839165994,1097326,IBM
2,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, MiFID Composite Cross Market Service",21475739068,79345498,IBMEUR.xbo
3,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, MiFID Composite Cross Market Service",21523029548,131770829,IBMUSD.xbo
4,QUOTExEQUITY,"International Business Machines Corp, Depository Receipt, Bolsa de Comercio de Buenos Aires",55838283585,1006058,IBM.BA
5,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, New York Stock Exchange",55838323096,1090370,IBM.N
6,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, London Stock Exchange",55836050580,1283576,IBMI.L
7,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NASDAQ INTERMARKET",55835333953,765744,IBM.TH
8,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, Cboe BZX Exchange",55845629228,53703996,IBM.Z
9,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, Cboe EDGX Exchange",21475059739,73533774,IBM.DG


In [9]:
# The 'name' specification will search across all names, such as common name, legal name, issuer name, etc. 
# Primarily used within a filter but can be used as a selectable property.
response=search.Definition(
    filter = "name eq 'IBM'",
).get_data()
response.data.df

,BusinessEntity,DocumentTitle,PermID,PI,RIC
0,ORGANISATION,"International Business Machines Corp, Public Company",<NA>,37036,<NA>
1,ORGANISATION,"Banco IBM SA, Private Company",<NA>,76208,<NA>
2,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",55839165994,1097326,IBM
3,ORGANISATION,"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",<NA>,18062670,<NA>
4,QUOTExEQUITY,"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",21481052421,48924732,0#IBMF:
5,QUOTExEQUITY,"Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam",21612423771,259118763,0#IBMDF:
6,QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex",21481052892,49450681,IBMFc1
7,QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex",21481053949,50092347,IBMFc2
8,QUOTExEQUITY,"Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam",21613372305,260213021,IBMDFc1
9,QUOTExEQUITY,"Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex",21481053950,50092348,IBMFc3


In [10]:
# The special character underscore (_) is a convenient token that can be used to select the default Properties.  
# This will allow users to quickly select the default values plus any other properties when testing out your search.
response=search.Definition(
    query = "IBM",
    top = 1000,
    select = "_, SearchAllCategoryv3"
).get_data()
response.data.df

,BusinessEntity,DocumentTitle,PermID,PI,RIC,SearchAllCategoryv3
0,ORGANISATION,"International Business Machines Corp, Public Company",<NA>,37036,<NA>,Companies/Issuers
1,ORGANISATION,"Banco IBM SA, Private Company",<NA>,76208,<NA>,Companies/Issuers
2,QUOTExEQUITY,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",55839165994,1097326,IBM,Equities
3,ORGANISATION,"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",<NA>,18062670,<NA>,Companies/Issuers
4,QUOTExEQUITY,"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",21481052421,48924732,0#IBMF:,Futures
...,...,...,...,...,...,...
995,QUOTExEQUITY,"OPRA International Business Machines Equity Option 127 Call Apr 2022 , Equity Cash Option, Call 127 USD 22-Apr-2022, OPRA",21877354993,589510405,IBMD222212700.U,Options
996,QUOTExEQUITY,"OPRA International Business Machines Equity Option 140 Call Apr 2022 , Equity Cash Option, Call 140 USD 22-Apr-2022, OPRA",21877355051,589510474,IBMD222214000.U,Options
997,QUOTExEQUITY,"OPRA International Business Machines Equity Option 120 Put Apr 2022 , Equity Cash Option, Put 120 USD 22-Apr-2022, OPRA",21877355116,589510530,IBMP222212000.U,Options
998,QUOTExEQUITY,"OPRA International Business Machines Equity Option 128 Put Apr 2022 , Equity Cash Option, Put 128 USD 22-Apr-2022, OPRA",21877355599,589511010,IBMP222212800.U,Options


#### Common Properties
As you become more familiar with search, you will soon recognize there will be a number of very useful properties that will act as your "go-tos" when filtering or selecting fields.

In [11]:
# DocumentTitle
# As you've seen in some of the above output, this property is on by default when you want to display your search results.  
# The DocumentTitle property is a quick way to describe the content retrieved for each matching document.  
# It is made up of the following child properties: 
#    'DTSubjectName', 'DTSimpleType', 'DTSource', and 'DTCharacteristics'.
response=search.Definition(
    query = "IBM",
    top = 100,
    select = "DocumentTitle, DTSubjectName, DTSimpleType, DTSource, DTCharacteristics"
).get_data()
response.data.df

,DocumentTitle,DTSubjectName,DTSimpleType,DTSource,DTCharacteristics
0,"International Business Machines Corp, Public Company",International Business Machines Corp,Public Company,<NA>,<NA>
1,"Banco IBM SA, Private Company",Banco IBM SA,Private Company,<NA>,<NA>
2,"International Business Machines Corp, Ordinary Share, NYSE Consolidated",International Business Machines Corp,Ordinary Share,NYSE Consolidated,<NA>
3,"Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company",Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4,Private Company,<NA>,<NA>
4,"Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex",Eurex International Business Machines Equity Future Chain Contract,Equity Future,Eurex,USD
...,...,...,...,...,...
95,"OPRA International Business Machines Equity Option 124 Call Mar 2022 , Equity Cash Option, Call 124 USD 25-Mar-2022, OPRA",OPRA International Business Machines Equity Option 124 Call Mar 2022,Equity Cash Option,OPRA,Call 124 USD 25-Mar-2022
96,"OPRA International Business Machines Equity Option 120 Put Mar 2022 , Equity Cash Option, Put 120 USD 25-Mar-2022, OPRA",OPRA International Business Machines Equity Option 120 Put Mar 2022,Equity Cash Option,OPRA,Put 120 USD 25-Mar-2022
97,"OPRA International Business Machines Equity Option 120 Call Mar 2022 , Equity Cash Option, Call 120 USD 25-Mar-2022, OPRA",OPRA International Business Machines Equity Option 120 Call Mar 2022,Equity Cash Option,OPRA,Call 120 USD 25-Mar-2022
98,"OPRA International Business Machines Equity Option 100 Call May 2022 , Equity Cash Option, Call 100 USD 20-May-2022, OPRA",OPRA International Business Machines Equity Option 100 Call May 2022,Equity Cash Option,OPRA,Call 100 USD 20-May-2022


In [12]:
# SearchAllCategoryv3
# When trying to narrow down your search to specific categories, this property can be extremely useful.  
# When you apply a Navigator on this property, it will return a list of ~40 categories, depending on the view. 
# Categories include asset-related instruments such as:
#     'Equities', 'Futures' , 'Options', 'Bonds', 'Commodities' 
#     but can also include things like: 'People', 'Analyst', 'Officers'.
response=search.Definition(
    query = "IBM",
    filter = "SearchAllCategoryv3 eq 'People'",
).get_data()
response.data.df

,BusinessEntity,DocumentTitle,PermID,PI
0,PERSON,John Denham - IsUnknown,34414661717,34414661717
1,PERSON,Charlie Maxfield - IsUnknown,34414669683,34414669683
2,PERSON,Claudia Barclay - IsUnknown,34414721056,34414721056
3,PERSON,Robert Ploder - IsUnknown,34414738667,34414738667
4,PERSON,Tom Cadigan - IsUnknown,34414743082,34414743082
5,PERSON,R. L. Jay Vivian - IsUnknown,34414808510,34414808510
6,PERSON,William E. Baker - IsUnknown,34415092341,34415092341


In [13]:
# CodeResolutionBase
# Represents the most significant codes defined within the document.  For example, when searching for a symbol such 
# as ISIN, SEDOL, RIC, etc, filtering against this property will significantly reduce the matches and only bring 
# back the most relevant results.
response=search.Definition(
    filter = "CodeResolutionBase eq 'US046353AQ14'",
    select = "DocumentTitle, ISIN, SEDOL, RIC"
).get_data()
response.data.df

,DocumentTitle,ISIN,SEDOL,RIC
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,BF4SNM7,046353AQ1=


In [14]:
# Conversely, if we were to choose the more generic term 'code', would result in a wider range of matches.
response=search.Definition(
    filter = "code eq 'US046353AQ14'",
    select = "DocumentTitle, ISIN, SEDOL, RIC"
).get_data()
response.data.df

,DocumentTitle,ISIN,SEDOL,RIC
0,"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,BF4SNM7,046353AQ1=
1,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS",<NA>,BF4SNM7,046353AQ1=1M
2,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS L2",<NA>,BF4SNM7,046353AQ1=2M
3,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, BARCLAYS TRADEWEB PRICE",<NA>,BF4SNM7,US046353AQ14=BXTW
4,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, CREDIT SUISSE TRADEWEB PRICE",<NA>,BF4SNM7,US046353AQ14=CSTW
5,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS COMPOSITE PLUS",<NA>,BF4SNM7,US046353AQ14=MKCP
6,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, RBC TRADEWEB PRICE",<NA>,BF4SNM7,US046353AQ14=RCTW
7,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEWEB APA",<NA>,BF4SNM7,US046353AQ14=TBEA
8,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TDS TRADEWEB PRICE",<NA>,BF4SNM7,US046353AQ14=TDTW
9,"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEWEB",<NA>,BF4SNM7,US046353AQ14=TE


#### Common Properties - RCS (Reuters Classification Scheme)
Throughout many documents, you will notice a number of RCS fields that are popular when selecting and filtering.

*Note* - I'll use navigators to demonstrate the values of these common properties.

In [16]:
# RCSAssetCategoryLeaf
# Specific to assets, this property will categorize the results into a readable identifier such as: 'Bond', 
# 'Commodity Future', 'Commercial Paper', etc.  Where the property SearchAllCategoryv3 provides general categories, 
# this property will be more specific to assets.  Pull out the top 10.
response = search.Definition(
    top = 0,
    navigators = "RCSAssetCategoryLeaf(buckets:20)"
).get_data()
response.data.raw["Navigators"]["RCSAssetCategoryLeaf"]["Buckets"]

[{'Label': 'Barrier Warrant', 'Count': 21086383},
 {'Label': 'Bond', 'Count': 17257741},
 {'Label': 'Commodity Future', 'Count': 4970044},
 {'Label': 'Traditional Warrant', 'Count': 4538892},
 {'Label': 'Equity Cash Option', 'Count': 3823099},
 {'Label': 'Interest Rate Future Spread', 'Count': 3162345},
 {'Label': 'Commodity Spread', 'Count': 2906954},
 {'Label': 'Equity Future', 'Count': 2826756},
 {'Label': 'Mortgage-Backed Security', 'Count': 2817290},
 {'Label': 'Certificate of Deposit', 'Count': 2384646},
 {'Label': 'Commercial Paper', 'Count': 2191816},
 {'Label': 'Warrant', 'Count': 2159748},
 {'Label': 'Commodity Future Option', 'Count': 2149361},
 {'Label': 'Commodity Spot', 'Count': 1970333},
 {'Label': 'Ordinary Share', 'Count': 1772729},
 {'Label': 'Commodity Forward', 'Count': 1643141},
 {'Label': 'Open-End Fund', 'Count': 1256037},
 {'Label': 'Buffered Risk / Bonus Certificate', 'Count': 1180000},
 {'Label': 'Commodity Cash Option', 'Count': 987691},
 {'Label': 'Equity In

In [17]:
# RCSCountryLeaf
# RCSIssuerCountryLeaf
# RCSExchangeCountryLeaf
# Specific to some assets, will allow you to narrow your search to focus on a specific country.  Pull out the top 20.
response = search.Definition(
    top = 0,
    navigators = "RCSCountryLeaf(buckets:20)"
).get_data()
response.data.raw["Navigators"]["RCSCountryLeaf"]["Buckets"]

[{'Label': 'United States', 'Count': 12921744},
 {'Label': 'Eurobond', 'Count': 4250915},
 {'Label': 'China (Mainland)', 'Count': 1283332},
 {'Label': 'Colombia', 'Count': 1063770},
 {'Label': 'South Korea', 'Count': 568259},
 {'Label': 'Germany', 'Count': 485013},
 {'Label': 'Canada', 'Count': 411927},
 {'Label': 'France', 'Count': 249586},
 {'Label': 'India', 'Count': 198460},
 {'Label': 'Japan', 'Count': 165052},
 {'Label': 'Italy', 'Count': 135134},
 {'Label': 'Chile', 'Count': 131491},
 {'Label': 'Australia', 'Count': 105527},
 {'Label': 'Brazil', 'Count': 67160},
 {'Label': 'Switzerland', 'Count': 66547},
 {'Label': 'Spain', 'Count': 62379},
 {'Label': 'Russia', 'Count': 59183},
 {'Label': 'Austria', 'Count': 58377},
 {'Label': 'Norway', 'Count': 57723},
 {'Label': 'United Kingdom', 'Count': 55415}]

In [18]:
# RCSCurrencyLeaf
# Specific to some assets, will allow you to narrow your search to focus on a specific currency.  Just pull out the top 20.
response = search.Definition(
    top = 0,
    navigators = "RCSCurrencyLeaf(buckets:20)"
).get_data()
response.data.raw["Navigators"]["RCSCurrencyLeaf"]["Buckets"]

[{'Label': 'Euro', 'Count': 34665617},
 {'Label': 'US Dollar', 'Count': 33249346},
 {'Label': 'Chinese Yuan', 'Count': 2397348},
 {'Label': 'US Cent', 'Count': 1545301},
 {'Label': 'Colombian Peso', 'Count': 1321492},
 {'Label': 'Japanese Yen', 'Count': 1135851},
 {'Label': 'Hong Kong Dollar', 'Count': 1102584},
 {'Label': 'Canadian Dollar', 'Count': 938110},
 {'Label': 'Swedish Krona', 'Count': 894207},
 {'Label': 'British Pound', 'Count': 853343},
 {'Label': 'Swiss Franc', 'Count': 816235},
 {'Label': 'Indian Rupee', 'Count': 805242},
 {'Label': 'South Korean Won', 'Count': 757079},
 {'Label': 'Australian Dollar', 'Count': 511188},
 {'Label': 'British Penny', 'Count': 464418},
 {'Label': 'Hungarian Forint', 'Count': 390777},
 {'Label': 'Norwegian Krone', 'Count': 385662},
 {'Label': 'South African Rand', 'Count': 312777},
 {'Label': 'Turkish Lira', 'Count': 297162},
 {'Label': 'Taiwanese Dollar', 'Count': 294460}]

In [19]:
# RCSTRBC2012Leaf
# Represents the industry sector.  For example, if I query for Apple Computers, I will get many different industry sectors such as: 
# 'Computer Hardware (NEC)', 'Laptop & Desktop Computers', 'Phones & Smart Phones', etc.
response = search.Definition(
    top = 0,
    navigators = "RCSTRBC2012Leaf(buckets:20)"
).get_data()
response.data.raw["Navigators"]["RCSTRBC2012Leaf"]["Buckets"]

[{'Label': 'Banks (NEC)', 'Count': 4323550},
 {'Label': 'Corporate Financial Services (NEC)', 'Count': 3119835},
 {'Label': 'Corporate Banks', 'Count': 1441807},
 {'Label': 'Retail & Mortgage Banks', 'Count': 1300378},
 {'Label': 'Investment Banking & Brokerage Services (NEC)', 'Count': 745595},
 {'Label': 'Public Finance Activities', 'Count': 669207},
 {'Label': 'Government & Government Finance (NEC)', 'Count': 419633},
 {'Label': 'Consumer Lending (NEC)', 'Count': 415425},
 {'Label': 'Investment Holding Companies (NEC)', 'Count': 384824},
 {'Label': 'Investment Management & Fund Operators (NEC)', 'Count': 356207},
 {'Label': 'Construction & Engineering (NEC)', 'Count': 342793},
 {'Label': 'Electric Utilities (NEC)', 'Count': 306845},
 {'Label': 'Wealth Management', 'Count': 299500},
 {'Label': 'Diversified Investment Services', 'Count': 290408},
 {'Label': 'Professional Information Services (NEC)', 'Count': 275760},
 {'Label': 'Real Estate Rental, Development & Operations (NEC)',
  '

In [20]:
# RCSTRBC2012Leaf
# RCSTRBC2012Genealogy
# RCSTRBC2012Name
# Provide a breakdown of the hierarchy defined for many of these RCS attributes.  You can see the mapping between the 
# hierarchy defined with the Genealogy property and Name property.  
# For example, the code 'B:262' maps to the name 'Banking & Investment Services'.
response = search.Definition(
    filter = "RCSTRBC2012Leaf eq 'Banks'",    # Limit the sector to banks so we can easily observe the output below
    top = 0,
    navigators = "RCSTRBC2012Leaf, RCSTRBC2012Genealogy, RCSTRBC2012Name"
).get_data()
response.data.raw

{'Total': 7119576,
 'Hits': [{'BusinessEntity': 'ORGANISATION',
   'DocumentTitle': 'JPMorgan Chase & Co, Public Company',
   'PI': '46434'},
  {'BusinessEntity': 'ORGANISATION',
   'DocumentTitle': 'Bank of America Corp, Public Company',
   'PI': '6074'},
  {'BusinessEntity': 'ORGANISATION',
   'DocumentTitle': 'Industrial and Commercial Bank of China Ltd, Public Company',
   'PI': '2317'},
  {'BusinessEntity': 'ORGANISATION',
   'DocumentTitle': 'Wells Fargo & Co, Public Company',
   'PI': '29083'},
  {'BusinessEntity': 'ORGANISATION',
   'DocumentTitle': 'China Merchants Bank Co Ltd, Public Company',
   'PI': '86932'},
  {'BusinessEntity': 'ORGANISATION',
   'DocumentTitle': 'China Construction Bank Corp, Public Company',
   'PI': '82217'},
  {'BusinessEntity': 'ORGANISATION',
   'DocumentTitle': 'Agricultural Bank of China Ltd, Public Company',
   'PI': '97893'},
  {'BusinessEntity': 'ORGANISATION',
   'DocumentTitle': 'Royal Bank of Canada, Public Company',
   'PI': '1196'},
  {'B

The above output includes 3 separate sections: RCSTRBC2012Genealogy, RCSTRBC2012Leaf and RCSTRBC2012Name.  For each you can see a breakdown ot the buckets.